# Jupyter Notebooklets Demo

### [@ianhellen](https://twitter.com/ianhellen)
#### Principal Dev - MSTIC, Azure Security

# What are notebooklets?

Collections of notebook cells that implement some useful reusable sequence

## Rationale
- Notebook code can quickly become complex and length:
  - Can obscure the information you are trying to display
  - Can be intimidating to non-developers
- Notebook code cells are not easily re-useable:
  - You can copy and paste but how do you sync changes back to original notebook?
  - Difficult to discover code snippets in notebooks
- Notebook code is often fragile:
  - Often not parameterized
  - Code blocks are frequently dependent on global values assigned earlier
  - Output data is not in any standard format
  - Difficult to test

## Characteristics of Notebooklets
- One or small number of entry points
- Must be paramertizable (e.g. you can supply hostname, IP Address, time range, etc.)
- Can query, process or visualize data (or any combination)
- Typically return a result or package of results for use later in the notebook


---
# Initializing the Notebook
Notebooklets depend on msticpy so we import/initialize this package.

In [1]:
import sys
import os
from IPython.display import display, HTML, Markdown

from msticpy.nbtools.nbinit import init_notebook
init_notebook(namespace=globals());

Processing imports....
Checking configuration....
Setting options....


---
# Notebooklets in use

## Import the package
- Discovers and imports notebooklet classes/modules

In [2]:
# pip install git+https://github.com/microsoft/msticnb

In [2]:
import msticnb as nb

4 notebooklets loaded.


---
## Intializing notebooklets - Calling init()
Before using any of the notebooklets you need to initialize the providers.

Providers are the libraries that do the work of fetching data from external sources that are
then used by the notebooklet code.

init() does the following:
- Loads required data providers
- Authenticates to providers if required at startup
- Can supply list of providers to load
- Can pass parameters to each provider (settings loaded from config by default)

In [3]:
nb.init?

Signature:
nb.init(
    query_provider:str='LogAnalytics',
    providers:Union[List[str], NoneType]=None,
    **kwargs,
)
Docstring:
Instantiate an instance of DataProviders.

Parameters
----------
query_provider : str, optional
    DataEnvironment name of the primary query provider.
    You can add addtional query providers by including them
    in the `providers` list.
providers : Optional[List[str]], optional
    A list of provider names, by default "LogAnalytics"

Other Parameters
----------------
kwargs
    You can pass parameters to individual providers using
    the following notation:
    `ProviderName_param_name="param_value"
    Where `ProviderName` is the name of the data provider,
    `param_name` is the parameter name expected by the
    provider and `param_value` is the value to assign to
    `param_name`. `param_value` can be any type.

    Depending on the provider, these parameters (with the
    prefix stripped) are sent to either the constructor or
    `connect` metho

### Data Providers
`list_providers()` gives a list of available providers
`get_def_providers()` shows you which providers are loaded by default

In [5]:
nb.DataProviders.list_providers()

['LogAnalytics',
 'AzureSentinel',
 'Kusto',
 'AzureSecurityCenter',
 'SecurityGraph',
 'MDATP',
 'LocalData',
 'azuredata',
 'tilookup',
 'geolitelookup',
 'ipstacklookup']

In [6]:
nb.DataProviders.get_def_providers()

['azuredata', 'tilookup', 'geolitelookup']

### Example - Running init, adding ipstacklookup to the default set of providers.
You can also prefix a provider name with "-" to remove it from the default set.

You can also specify an explicit list of providers to override the defaults entirely. E.g
```
nb.init(query_provider="AzureSentinel", providers=["ipstacklookup", "tilookup"])
```

> **Note** you cannot mix the "+"/"-" with un-prefixed provider names.
> Doing this will cause an error to be thrown.
> e.g. <br>
> `nb.init(query_provider="AzureSentinel", providers=["+ipstacklookup", "tilookup"])`
> <br>is illegal.

In [7]:
nb.init(query_provider="AzureSentinel", providers=["-tilookup", "+ipstacklookup"])

Please wait. Loading Kqlmagic extension...


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Loaded providers: LogAnalytics, geolitelookup, ipstacklookup


---
## Using LocalData Provider
The LocalData provider allows you to substitue local files for queries that you normally
make to online data sources such as AzureSentinel.

When we call init() we use the "LocalData_" prefix to pass the "query_paths" and "data_paths"
parameters to the underlying provider.

- Specify a folder where data files are stored with `LocalData_data_paths` (list[str])
- Specify a folder containing query definition files `LocalData_query_paths` (list[str])


In notebooklets queries are available as self.query_provider

In [4]:
nb.init(
    "LocalData", providers=["-tilookup"],
    LocalData_data_paths=["e:\\src\\msticnb\\msticnb\\tests\\testdata"],
    LocalData_query_paths=["e:\\src\\msticnb\\msticnb\\tests\\testdata"],
)

e:\src\msticnb\msticnb\tests\testdata\msticpyconfig-test.yaml is not a valid query definition file - skipping.
e:\src\msticnb\msticnb\tests\testdata\custom_nb\host\host_test_nb.yaml is not a valid query definition file - skipping.


Loaded providers: LocalData, geolitelookup


### Test the provider by making a query

In [5]:
# Loaded providers are available in the data_providers dictionary
qry_prov = nb.data_providers["LocalData"]

# This is a normal query provider with attached queries
qry_prov.WindowsSecurity.list_host_events().head()

,TenantId,TimeGenerated,SourceSystem,Account,AccountType,Computer,EventSourceName,Channel,Task,Level,EventData,EventID,Activity,PartitionKey,RowKey,StorageAccount,AzureDeploymentID,AzureTableName,AccessList,AccessMask,AccessReason,AccountDomain,AccountExpires,AccountName,AccountSessionIdentifier,...,TargetUserSid,TemplateContent,TemplateDSObjectFQDN,TemplateInternalName,TemplateOID,TemplateSchemaVersion,TemplateVersion,TokenElevationType,TransmittedServices,UserAccountControl,UserParameters,UserPrincipalName,UserWorkstations,VirtualAccount,VendorIds,Workstation,WorkstationName,SourceComputerId,EventOriginId,MG,TimeCollected,ManagementGroupName,Type,_ResourceId,EventProperties
0,52b1ab41-869e-4138-9e40-2a4457f09bf0,2019-02-11 08:40:22.537,OpsManager,WORKGROUP\MSTICAlertsWin1$,Machine,MSTICAlertsWin1,Microsoft-Windows-Security-Auditing,Security,13826,8,"<EventData xmlns=""http://schemas.microsoft.com/win/2004/08/events/event"">\r\n <Data Name=""Targe...",4799,4799 - A security-enabled local group membership was enumerated,,,,,,,,,,,,,...,,,,,,,,,,,,,,,,,,263a788b-6526-4cdc-8ed9-d79402fe4aa0,d0ec0118-9b6b-477e-ba9f-e4ead3665884,00000000-0000-0000-0000-000000000001,2019-02-11 08:40:40.387,AOI-52b1ab41-869e-4138-9e40-2a4457f09bf0,SecurityEvent,/subscriptions/40dcc8bf-0478-4f3b-b275-ed0a94f2c013/resourcegroups/asihuntomsworkspacerg/provide...,"{'TargetUserName': 'Remote Desktop Users', 'TargetDomainName': 'Builtin', 'TargetSid': 'S-1-5-32..."
1,52b1ab41-869e-4138-9e40-2a4457f09bf0,2019-02-11 08:40:24.827,OpsManager,WORKGROUP\MSTICAlertsWin1$,Machine,MSTICAlertsWin1,Microsoft-Windows-Security-Auditing,Security,13826,8,"<EventData xmlns=""http://schemas.microsoft.com/win/2004/08/events/event"">\r\n <Data Name=""Targe...",4799,4799 - A security-enabled local group membership was enumerated,,,,,,,,,,,,,...,,,,,,,,,,,,,,,,,,263a788b-6526-4cdc-8ed9-d79402fe4aa0,5365f041-1345-4cd0-8fbb-41ccbaa16a39,00000000-0000-0000-0000-000000000001,2019-02-11 08:40:40.387,AOI-52b1ab41-869e-4138-9e40-2a4457f09bf0,SecurityEvent,/subscriptions/40dcc8bf-0478-4f3b-b275-ed0a94f2c013/resourcegroups/asihuntomsworkspacerg/provide...,"{'TargetUserName': 'Administrators', 'TargetDomainName': 'Builtin', 'TargetSid': 'S-1-5-32-544',..."
2,52b1ab41-869e-4138-9e40-2a4457f09bf0,2019-02-11 08:40:24.827,OpsManager,WORKGROUP\MSTICAlertsWin1$,Machine,MSTICAlertsWin1,Microsoft-Windows-Security-Auditing,Security,13824,8,"<EventData xmlns=""http://schemas.microsoft.com/win/2004/08/events/event"">\r\n <Data Name=""Targe...",4798,4798 - A user's local group membership was enumerated.,,,,,,,,,,,,,...,,,,,,,,,,,,,,,,,,263a788b-6526-4cdc-8ed9-d79402fe4aa0,5333954d-9157-443c-9657-6ef390c73abb,00000000-0000-0000-0000-000000000001,2019-02-11 08:40:40.387,AOI-52b1ab41-869e-4138-9e40-2a4457f09bf0,SecurityEvent,/subscriptions/40dcc8bf-0478-4f3b-b275-ed0a94f2c013/resourcegroups/asihuntomsworkspacerg/provide...,"{'TargetUserName': 'MSTICAdmin', 'TargetDomainName': 'MSTICAlertsWin1', 'TargetSid': 'S-1-5-21-9..."
3,52b1ab41-869e-4138-9e40-2a4457f09bf0,2019-02-11 08:40:24.833,OpsManager,WORKGROUP\MSTICAlertsWin1$,Machine,MSTICAlertsWin1,Microsoft-Windows-Security-Auditing,Security,13824,8,"<EventData xmlns=""http://schemas.microsoft.com/win/2004/08/events/event"">\r\n <Data Name=""Targe...",4798,4798 - A user's local group membership was enumerated.,,,,,,,,,,,,,...,,,,,,,,,,,,,,,,,,263a788b-6526-4cdc-8ed9-d79402fe4aa0,d2fded8a-e507-4017-aedd-cb6da6a2b624,00000000-0000-0000-0000-000000000001,2019-02-11 08:40:40.387,AOI-52b1ab41-869e-4138-9e40-2a4457f09bf0,SecurityEvent,/subscriptions/40dcc8bf-0478-4f3b-b275-ed0a94f2c013/resourcegroups/asihuntomsworkspacerg/provide...,"{'TargetUserName': 'DefaultAccount', 'TargetDomainName': 'MSTICAlertsWin1', 'TargetSid': 'S-1-5-..."
4,52b1ab41-869e-4138-9e40-2a4457f09bf0,2019-02-11 08:40:24.833,OpsManager,WORKGROUP\MSTICAlertsWin1$,Machine,MSTICAlertsWin1,Microsoft-Windows-Security-Auditing,Security,13824,8,"<EventData xmlns=""http://schemas.microsoft.com/wi

---
## Notebooklet classes are discovered and imported at load time

Although you can manually initiate a a run to read more notebooklets.

#### The `nblts` attribute exposes notebooklets (niblets?) in a tree structure
- Useful for autocomplete when you more or less know what you're looking for

The top level in the hierarchy is the data environment (e.g. azsent == AzureSentinel). Beneath these the notebooklets are grouped into various categories such as host, network, etc.

In [10]:
print(nb.nblts)

azsent
  host
    HostSummary (Notebooklet)
    WinHostEvents (Notebooklet)
  network
    NetworkFlowSummary (Notebooklet)



Access an individual notebook using this path structure

In [11]:
nb.nblts.azsent.host.HostSummary?

Init signature:
nb.nblts.azsent.host.HostSummary(
    data_providers:Union[<msticnb.data_providers.SingletonDecorator object at 0x000002535346E630>, NoneType]=None,
    **kwargs,
)
Docstring:     
HostSummary Notebooklet class.

Queries and displays information about a host including:

- IP address assignment
- Related alerts
- Related hunting/investigation bookmarks
- Azure subscription/resource data.


Default Options
---------------
heartbeat: Query Heartbeat table for host information.
azure_net: Query AzureNetworkAnalytics table for host network topology information.
alerts: Query any alerts for the host.
bookmarks: Query any bookmarks for the host.
azure_api: Query Azure API for VM information.

Other Options
-------------
None
Init docstring:
Intialize a new instance of the notebooklet class.

Parameters
----------
data_providers : DataProviders, Optional
    Optional DataProviders instance to query data.
    Most classes require this.

Raises
------
MsticnbDataProviderError
   

### Notebooklets are exposed in `nb.nb_index`
The values reflect the physical path in which the notebooklets are stored (you can ignore this)

In [12]:
nb.nb_index

{'nblts.azsent.host.HostSummary': msticnb.nb.azsent.host.host_summary.HostSummary,
 'nblts.azsent.host.WinHostEvents': msticnb.nb.azsent.host.win_host_events.WinHostEvents,
 'nblts.azsent.network.NetworkFlowSummary': msticnb.nb.azsent.network.network_flow_summary.NetworkFlowSummary}

## There is a find function that looks for:
- text or regulate expressions
- searches class docstring
- metadata such as entities supported and options supported

In [13]:
nb.find("host, net.*", full_match=True)

[('HostSummary', msticnb.nb.azsent.host.host_summary.HostSummary),
 ('NetworkFlowSummary',
  msticnb.nb.azsent.network.network_flow_summary.NetworkFlowSummary)]

---
# More detailed (and user-friendly) help in the `show_help()` method

In [6]:
nb.nblts.azsent.host.HostSummary.show_help()

---
# How are notebooklets used?

## Most require time range parameters

Usually the notebooklet also the ID of the entity that you're running the notebooklet for. For example, a host name, an IP Address, etc.

Some notebooklets process data in the form of a dataframe. Use the `data` parameter to pass this.

> **Note** You can also pass other parameters used by the notebooklet as keyword arguments (`**kwargs`)

In [15]:
time_span = nbwidgets.QueryTime(auto_display=True, units="day", origin_time=pd.to_datetime("2019-02-10"), before=10)
from msticnb.common import TimeSpan

HTML(value='<h4>Set query time boundaries</h4>')

## Run the notebooklet using the `run()` method

>  **Note:** You'll want to assign the return value of `run()` to something or terminate with a semicolon<br>
>  Both the notebooklet and the return `result` class generate displayable output - so you'll get
>  a lot of duplicated output.

In [16]:
host_summary = nb.nblts.azsent.host.HostSummary()
host_sum_rslt = host_summary.run(value="Msticalertswin1", timespan=(time_span.start, time_span.end))

AttributeError: 'HostSummary' object has no attribute 'NBMetadata'

## Result classes content can be displayed in the notebook
Use `display(result)` if you want to display the content in the middle of a cell

In [16]:
host_sum_rslt

,TenantId,TimeGenerated,AlertDisplayName,AlertName,Severity,Description,ProviderName,VendorName,VendorOriginalId,SystemAlertId,ResourceId,SourceComputerId,AlertType,ConfidenceLevel,ConfidenceScore,IsIncident,StartTimeUtc,EndTimeUtc,ProcessingEndTime,RemediationSteps,ExtendedProperties,Entities,SourceSystem,WorkspaceSubscriptionId,WorkspaceResourceGroup,ExtendedLinks,ProductName,ProductComponentName,AlertLink,Type,CompromisedEntity
0,52b1ab41-869e-4138-9e40-2a4457f09bf0,2019-02-18 02:29:07,SSH Anomalous Login ML,SSH Anomalous Login ML,Low,Anomalous login detected for SSH account,CustomAlertRule,Alert Rule,b0e143b8-4fa8-47bc-8bc1-9780c8b75541,f1ce87ca-8863-4a66-a0bd-a4d3776a7c64,,,CustomAlertRule_0a4e5f7c-9756-45f8-83c4-94c756844698,Unknown,NaN,False,2019-02-18 01:49:02,2019-02-18 02:19:02,2019-02-18 02:29:07,,"{\r\n ""Alert Mode"": ""Aggregated"",\r\n ""Search Query"": ""{\""detailBladeInputs\"":{\""id\"":\""/subsc...","[\r\n {\r\n ""$id"": ""3"",\r\n ""Address"": ""23.97.60.214"",\r\n ""Type"": ""ip"",\r\n ""Count...",Detection,40dcc8bf-0478-4f3b-b275-ed0a94f2c013,asihuntomsworkspacerg,,,,,SecurityAlert,
1,52b1ab41-869e-4138-9e40-2a4457f09bf0,2019-02-18 01:59:09,SSH Anomalous Login ML,SSH Anomalous Login ML,Low,Anomalous login detected for SSH account,CustomAlertRule,Alert Rule,4f454388-02d3-4ace-98bf-3a7e4fdef361,3968ef4e-b322-48ca-b297-e984aff8888d,,,CustomAlertRule_0a4e5f7c-9756-45f8-83c4-94c756844698,Unknown,NaN,False,2019-02-18 01:19:02,2019-02-18 01:49:02,2019-02-18 01:59:09,,"{\r\n ""Alert Mode"": ""Aggregated"",\r\n ""Search Query"": ""{\""detailBladeInputs\"":{\""id\"":\""/subsc...","[\r\n {\r\n ""$id"": ""3"",\r\n ""Address"": ""203.0.113.1"",\r\n ""Type"": ""ip"",\r\n ""Count""...",Detection,40dcc8bf-0478-4f3b-b275-ed0a94f2c013,asihuntomsworkspacerg,,,,,SecurityAlert,
2,52b1ab41-869e-4138-9e40-2a4457f09bf0,2019-02-18 02:29:07,SSH Anomalous Login ML,SSH Anomalous Login ML,Low,Anomalous login detected for SSH account,CustomAlertRule,Alert Rule,b0e143b8-4fa8-47bc-8bc1-9780c8b75541,3a78a119-abe9-4b5e-9786-300ddcfd9530,,,CustomAlertRule_0a4e5f7c-9756-45f8-83c4-94c756844698,Unknown,NaN,False,2019-02-18 01:49:02,2019-02-18 02:19:02,2019-02-18 02:29:07,,"{\r\n ""Alert Mode"": ""Aggregated"",\r\n ""Search Query"": ""{\""detailBladeInputs\"":{\""id\"":\""/subsc...","[\r\n {\r\n ""$id"": ""3"",\r\n ""Address"": ""23.97.60.214"",\r\n ""Type"": ""ip"",\r\n ""Count...",Detection,40dcc8bf-0478-4f3b-b275-ed0a94f2c013,asihuntomsworkspacerg,,,,,SecurityAlert,
3,52b1ab41-869e-4138-9e40-2a4457f09bf0,2019-02-18 02:43:27,SSH Anomalous Login ML,SSH Anomalous Login ML,Low,Anomalous login detected for SSH account,CustomAlertRule,Alert Rule,3f27593a-db5b-4ef4-bdc5-f6ce1915f496,8f622935-1422-41e6-b8f6-9119e681645c,,,CustomAlertRule_0a4e5f7c-9756-45f8-83c4-94c756844698,Unknown,NaN,False,2019-02-18 01:33:19,2019-02-18 02:33:19,2019-02-18 02:43:27,,"{\r\n ""Alert Mode"": ""Aggregated"",\r\n ""Search Query"": ""{\""detailBladeInputs\"":{\""id\"":\""/subsc...","[\r\n {\r\n ""$id"": ""3"",\r\n ""Address"": ""23.97.60.214"",\r\n ""Type"": ""ip"",\r\n ""Count...",Detection,40dcc8bf-0478-4f3b-b275-ed0a94f2c013,asihuntomsworkspacerg,,,,,SecurityAlert,
4,52b1ab41-869e-4138-9e40-2a4457f09bf0,2019-02-18 01:54:11,SSH Anomalous Login ML,SSH Anomalous Login ML,Low,Anomalous login detected for SSH account,CustomAlertRule,Alert Rule,3cbe0028-14e8-43ad-8dc2-77c96d8bb015,64a2b4af-c3d7-422c-820b-7f1feb664222,,,CustomAlertRule_0a4e5f7c-9756-45f8-83c4-94c756844698,Unknown,NaN,False,2019-02-18 01:14:02,2019-02-18 01:44:02,2019-02-18 01:54:11,,"{\r\n ""Alert Mode"": ""Aggregated"",\r\n ""Search Query"": ""{\""detailBladeInputs\"":{\""id\"":\""/subsc...","[\r\n {\r\n ""$id"": ""3"",\r\n ""Address"": ""203.0.113.1"",\r\n ""Type"": ""ip"",\r\n ""Count""...",Detection,40dcc8bf-0478-4f3b-b275-ed0a94f2c013,asihuntomsworkspacerg,,,,,SecurityAlert,
,TenantId,TimeGenerated,AlertDisplayName,AlertName,Severity,Description,ProviderName,VendorName,VendorOriginalId,SystemAlertId,R

---
## Simple Notebooklet browser

In [17]:
nb.browse()

In [19]:
# value="MSTICAlertsWin1", timespan=time_span

win_host_events = nb.nblts.azsent.host.WinHostEvents()
timespan = TimeSpan(start="2020-05-07 00:10")
win_host_events_rslt = win_host_events.run(value="MSTICAlertsWin1", timespan=timespan)

Getting data from SecurityEvent...


Activity,DWM-1,DWM-2,IUSR,LOCAL SERVICE,MSTICAdmin,MSTICAlertsWin1$,NETWORK SERVICE,No Account,SYSTEM,ian
1100 - The event logging service has shut down.,0,0,0,0,0,0,0,1,0,0
4608 - Windows is starting up.,0,0,0,0,0,0,0,1,0,0
4616 - The system time was changed.,0,0,0,2,0,0,0,0,0,0
4625 - An account failed to log on.,0,0,0,0,0,0,0,0,0,2
4634 - An account was logged off.,0,4,0,0,12,0,0,0,0,2
4647 - User initiated logoff.,0,0,0,0,2,0,0,0,0,0
4648 - A logon was attempted using explicit credentials.,0,0,0,0,0,10,0,0,0,0
4672 - Special privileges assigned to new logon.,2,2,1,1,14,0,1,0,60,0
4720 - A user account was created.,0,0,0,0,2,0,0,0,0,0
4722 - A user account was enabled.,0,0,0,0,2,0,0,0,0,0


Activity,MSTICAdmin
4720 - A user account was created.,2
4722 - A user account was enabled.,2
4724 - An attempt was made to reset an account's password.,4
4726 - A user account was deleted.,2
4728 - A member was added to a security-enabled global group.,2
4729 - A member was removed from a security-enabled global group.,2
4732 - A member was added to a security-enabled local group.,4
4733 - A member was removed from a security-enabled local group.,3
4738 - A user account was changed.,5


Loading BokehJS ...

## Additional operations apart from `run()`
We can use expand events to unpack the `EventData` column for selected EventIDs

In [20]:
win_host_events_rslt.account_events.head(5)

,TenantId,TimeGenerated,SourceSystem,Account,AccountType,Computer,EventSourceName,Channel,Task,Level,EventData,EventID,Activity,PartitionKey,RowKey,StorageAccount,AzureDeploymentID,AzureTableName,AccessList,AccessMask,AccessReason,AccountDomain,AccountExpires,AccountName,AccountSessionIdentifier,...,TargetUserSid,TemplateContent,TemplateDSObjectFQDN,TemplateInternalName,TemplateOID,TemplateSchemaVersion,TemplateVersion,TokenElevationType,TransmittedServices,UserAccountControl,UserParameters,UserPrincipalName,UserWorkstations,VirtualAccount,VendorIds,Workstation,WorkstationName,SourceComputerId,EventOriginId,MG,TimeCollected,ManagementGroupName,Type,_ResourceId,EventProperties
47,52b1ab41-869e-4138-9e40-2a4457f09bf0,2019-02-11 09:58:50.173,OpsManager,MSTICAlertsWin1\MSTICAdmin,User,MSTICAlertsWin1,Microsoft-Windows-Security-Auditing,Security,13826,8,"<EventData xmlns=""http://schemas.microsoft.com/win/2004/08/events/event"">\r\n <Data Name=""Membe...",4728,4728 - A member was added to a security-enabled global group.,,,,,,,,,,,,,...,,,,,,,,,,,,,,,,,,263a788b-6526-4cdc-8ed9-d79402fe4aa0,27df6071-1e81-4e24-934c-dc96667b83ab,00000000-0000-0000-0000-000000000001,2019-02-11 09:58:51.400,AOI-52b1ab41-869e-4138-9e40-2a4457f09bf0,SecurityEvent,/subscriptions/40dcc8bf-0478-4f3b-b275-ed0a94f2c013/resourcegroups/asihuntomsworkspacerg/provide...,"{'MemberName': '-', 'MemberSid': 'S-1-5-21-996632719-2361334927-4038480536-1118', 'TargetUserNam..."
48,52b1ab41-869e-4138-9e40-2a4457f09bf0,2019-02-11 09:58:50.173,OpsManager,MSTICAlertsWin1\MSTICAdmin,User,MSTICAlertsWin1,Microsoft-Windows-Security-Auditing,Security,13824,8,"<EventData xmlns=""http://schemas.microsoft.com/win/2004/08/events/event"">\r\n <Data Name=""Targe...",4720,4720 - A user account was created.,,,,,,,,,,%%1794,,,...,,,,,,,,,,\t\t%%2080 \t\t%%2082 \t\t%%2084,%%1793,-,%%1793,,,,,263a788b-6526-4cdc-8ed9-d79402fe4aa0,2c09036a-5ca7-4115-9ddf-e9eb49c14247,00000000-0000-0000-0000-000000000001,2019-02-11 09:58:51.400,AOI-52b1ab41-869e-4138-9e40-2a4457f09bf0,SecurityEvent,/subscriptions/40dcc8bf-0478-4f3b-b275-ed0a94f2c013/resourcegroups/asihuntomsworkspacerg/provide...,"{'TargetUserName': 'abai$', 'TargetDomainName': 'MSTICAlertsWin1', 'TargetSid': 'S-1-5-21-996632..."
49,52b1ab41-869e-4138-9e40-2a4457f09bf0,2019-02-11 09:58:50.183,OpsManager,MSTICAlertsWin1\MSTICAdmin,User,MSTICAlertsWin1,Microsoft-Windows-Security-Auditing,Security,13824,8,"<EventData xmlns=""http://schemas.microsoft.com/win/2004/08/events/event"">\r\n <Data Name=""Targe...",4722,4722 - A user account was enabled.,,,,,,,,,,,,,...,,,,,,,,,,,,,,,,,,263a788b-6526-4cdc-8ed9-d79402fe4aa0,fefd6761-e431-4cfa-9cd2-c5700f6186df,00000000-0000-0000-0000-000000000001,2019-02-11 09:58:51.400,AOI-52b1ab41-869e-4138-9e40-2a4457f09bf0,SecurityEvent,/subscriptions/40dcc8bf-0478-4f3b-b275-ed0a94f2c013/resourcegroups/asihuntomsworkspacerg/provide...,"{'TargetUserName': 'abai$', 'TargetDomainName': 'MSTICAlertsWin1', 'TargetSid': 'S-1-5-21-996632..."
50,52b1ab41-869e-4138-9e40-2a4457f09bf0,2019-02-11 09:58:50.183,OpsManager,MSTICAlertsWin1\MSTICAdmin,User,MSTICAlertsWin1,Microsoft-Windows-Security-Auditing,Security,13824,8,"<EventData xmlns=""http://schemas.microsoft.com/win/2004/08/events/event"">\r\n <Data Name=""Dummy...",4738,4738 - A user account was changed.,,,,,,,,,,%%1794,,,...,,,,,,,,,,\t\t%%2048 \t\t%%2050,-,-,%%1793,,,,,263a788b-6526-4cdc-8ed9-d79402fe4aa0,1d3997a3-9ede-4f9b-877a-eaabc63a3c1e,00000000-0000-0000-0000-000000000001,2019-02-11 09:58:51.400,AOI-52b1ab41-869e-4138-9e40-2a4457f09bf0,SecurityEvent,/subscriptions/40dcc8bf-0478-4f3b-b275-ed0a94f2c013/resourcegroups/asihuntomsworkspacerg/provide...,"{'Dummy': '-', 'TargetUserName': 'abai$', 'TargetDomainName': 'MSTICAlertsWin1', 'TargetSid': 'S..."
51,52b1ab41-869e-4138-9e40-2a4457f09bf0,2019-02-11 09:58:50.183,OpsManager,MSTICAlertsWin1\MSTICAdmin,User,MSTICAlertsWin1,Microsoft-Windows-Security-Auditing,Security,13824,8,"<EventData xmlns=""http://schemas.microsoft.c

In [21]:
win_host_events.expand_events(event_ids=4728).head(5)

,TenantId,TimeGenerated,SourceSystem,Account,AccountType,Computer,EventSourceName,Channel,Task,Level,EventData,EventID,Activity,MemberName,MemberSid,PrivilegeList,SubjectAccount,SubjectDomainName,SubjectLogonId,SubjectUserName,SubjectUserSid,TargetAccount,TargetDomainName,TargetSid,TargetUserName,SourceComputerId,EventOriginId,MG,TimeCollected,ManagementGroupName,Type,_ResourceId
47,52b1ab41-869e-4138-9e40-2a4457f09bf0,2019-02-11 09:58:50.173,OpsManager,MSTICAlertsWin1\MSTICAdmin,User,MSTICAlertsWin1,Microsoft-Windows-Security-Auditing,Security,13826,8,"<EventData xmlns=""http://schemas.microsoft.com/win/2004/08/events/event"">\r\n <Data Name=""Membe...",4728,4728 - A member was added to a security-enabled global group.,-,S-1-5-21-996632719-2361334927-4038480536-1118,-,MSTICAlertsWin1\MSTICAdmin,MSTICAlertsWin1,0xbd57571,MSTICAdmin,S-1-5-21-996632719-2361334927-4038480536-500,MSTICAlertsWin1\None,MSTICAlertsWin1,S-1-5-21-996632719-2361334927-4038480536-513,None,263a788b-6526-4cdc-8ed9-d79402fe4aa0,27df6071-1e81-4e24-934c-dc96667b83ab,00000000-0000-0000-0000-000000000001,2019-02-11 09:58:51.400,AOI-52b1ab41-869e-4138-9e40-2a4457f09bf0,SecurityEvent,/subscriptions/40dcc8bf-0478-4f3b-b275-ed0a94f2c013/resourcegroups/asihuntomsworkspacerg/provide...
58,52b1ab41-869e-4138-9e40-2a4457f09bf0,2019-02-11 09:58:50.447,OpsManager,MSTICAlertsWin1\MSTICAdmin,User,MSTICAlertsWin1,Microsoft-Windows-Security-Auditing,Security,13826,8,"<EventData xmlns=""http://schemas.microsoft.com/win/2004/08/events/event"">\r\n <Data Name=""Membe...",4728,4728 - A member was added to a security-enabled global group.,-,S-1-5-21-996632719-2361334927-4038480536-1119,-,MSTICAlertsWin1\MSTICAdmin,MSTICAlertsWin1,0xbd57571,MSTICAdmin,S-1-5-21-996632719-2361334927-4038480536-500,MSTICAlertsWin1\None,MSTICAlertsWin1,S-1-5-21-996632719-2361334927-4038480536-513,None,263a788b-6526-4cdc-8ed9-d79402fe4aa0,73b0fe4e-9886-43ab-afa6-b43eb7434402,00000000-0000-0000-0000-000000000001,2019-02-11 09:58:51.400,AOI-52b1ab41-869e-4138-9e40-2a4457f09bf0,SecurityEvent,/subscriptions/40dcc8bf-0478-4f3b-b275-ed0a94f2c013/resourcegroups/asihuntomsworkspacerg/provide...


---
# Anatomy of a Notebooklet

# Three sections:
- Results class - what is it going to return
- Notebooklet class - `run()` defines what the notebooklet does
- Code - series of functions that do the actual work

In [22]:
nb.nblts.azsent.host.WinHostEvents.import_cell()

In [ ]:
# -------------------------------------------------------------------------
# Copyright (c) Microsoft Corporation. All rights reserved.
# Licensed under the MIT License. See License.txt in the project root for
# license information.
# --------------------------------------------------------------------------
"""Notebooklet for Windows Security Events."""
import pkgutil
import os
from typing import Any, Optional, Iterable, Union, Dict
from defusedxml import ElementTree
from defusedxml.ElementTree import ParseError

import attr
from bokeh.plotting.figure import Figure
from bokeh.models import LayoutDOM
from IPython.display import display
import numpy as np
import pandas as pd
from msticpy.nbtools import nbdisplay

from msticnb.common import (
    TimeSpan,
    MsticnbMissingParameterError,
    nb_data_wait,
    set_text,
    nb_markdown,
)
from msticnb.notebooklet import Notebooklet, NotebookletResult, NBMetaData
from msticnb. import nb_metadata

from msticnb._version import VERSION

__version__ = VERSION
__author__ = "Ian Hellen"


_CLS_METADATA: NBMetaData
_CELL_DOCS: Dict[str, Any]
_CLS_METADATA, _CELL_DOCS = nb_metadata.read_mod_metadata(__file__, __name__)


# pylint: disable=too-few-public-methods
@attr.s(auto_attribs=True)
class WinHostEventsResult(NotebookletResult):
    """
    Windows Host Security Events Results.

    Attributes
    ----------
    all_events : pd.DataFrame
        DataFrame of all raw events retrieved.
    event_pivot : pd.DataFrame
        DataFrame that is a pivot table of event ID
        vs. Account
    account_events : pd.DataFrame
        DataFrame containing a subset of account management
        events such as account and group modification.
    acct_pivot : pd.DataFrame
        DataFrame that is a pivot table of event ID
        vs. Account of account management events
    account_timeline : Union[Figure, LayoutDOM]
        Bokeh plot figure or Layout showing the account events on an
        interactive timeline.
    expanded_events : pd.DataFrame
        If `expand_events` option is specified, this will contain
        the parsed/expanded EventData as individual columns.

    """

    description: str = "Windows Host Security Events"
    all_events: pd.DataFrame = None
    event_pivot: pd.DataFrame = None
    account_events: pd.DataFrame = None
    account_pivot: pd.DataFrame = None
    account_timeline: Union[Figure, LayoutDOM] = None
    expanded_events: pd.DataFrame = None


class WinHostEvents(Notebooklet):
    """
    Windows host Security Events Notebooklet class.

    Queries and displays Windows Security Events including:

    - All security events summary
    - Extracting and displaying account management events
    - Account management event timeline
    - Optionally parsing packed event data into DataFrame columns

    Process (4688) and Account Logon (4624, 4625) are not included
    in the event types processed by this module.

    """

    metadata = _CLS_METADATA
    __doc__ = nb_metadata.update_class_doc(__doc__, metadata)
    _cell_docs = _CELL_DOCS

    @set_text(docs=_CELL_DOCS, key="run")
    def run(
        self,
        value: Any = None,
        data: Optional[pd.DataFrame] = None,
        timespan: Optional[TimeSpan] = None,
        options: Optional[Iterable[str]] = None,
        **kwargs,
    ) -> WinHostEventsResult:
        """
        Return Windows Security Event summary.

        Parameters
        ----------
        value : str
            Host name
        data : Optional[pd.DataFrame], optional
            Not used, by default None
        timespan : TimeSpan
            Timespan over which operations such as queries will be
            performed, by default None.
            This can be a TimeStamp object or another object that
            has valid `start`, `end`, or `period` attributes.
        options : Optional[Iterable[str]], optional
            List of options to use, by default None.
            A value of None means use default options.
            Options prefixed with "+" will be added to the default options.
            To see the list of available options type `help(cls)` where
            "cls" is the notebooklet class or an instance of this class.

        Other Parameters
        ----------------
        start : Union[datetime, datelike-string]
            Alternative to specifying timespan parameter.
        end : Union[datetime, datelike-string]
            Alternative to specifying timespan parameter.

        Returns
        -------
        HostSummaryResult
            Result object with attributes for each result type.

        Raises
        ------
        MsticnbMissingParameterError
            If required parameters are missing

        """
        super().run(
            value=value, data=data, timespan=timespan, options=options, **kwargs
        )

        if not value:
            raise MsticnbMissingParameterError("value")
        if not timespan:
            raise MsticnbMissingParameterError("timespan.")

        result = WinHostEventsResult()
        result.description = self.metadata.description
        result.timespan = timespan

        all_events_df, event_pivot_df = _get_win_security_events(
            self.query_provider, host_name=value, timespan=self.timespan
        )
        result.all_events = all_events_df
        result.event_pivot = event_pivot_df

        if "event_pivot" in self.options:
            _display_event_pivot(event_pivot=event_pivot_df)

        if "acct_events" in self.options:
            result.account_events = _extract_acct_mgmt_events(event_data=all_events_df)
            result.account_pivot = _create_acct_event_pivot(
                account_event_data=result.account_events
            )
            if result.account_pivot is not None:
                _display_acct_event_pivot(event_pivot_df=result.account_pivot)
                result.account_timeline = _display_acct_mgmt_timeline(
                    acct_event_data=result.account_events
                )

        if "expand_events" in self.options:
            result.expanded_events = _parse_eventdata(all_events_df)

        nb_markdown("To unpack eventdata from selected events use expand_events()")
        self._last_result = result  # pylint: disable=attribute-defined-outside-init
        return self._last_result

    def expand_events(
        self, event_ids: Optional[Union[int, Iterable[int]]] = None
    ) -> pd.DataFrame:
        """
        Expand `EventData` for `event_ids` into separate columns.

        Parameters
        ----------
        event_ids : Optional[Union[int, Iterable[int]]], optional
            Single or interable of event IDs (ints).
            If no event_ids are specified all events will be expanded.

        Returns
        -------
        pd.DataFrame
            Results with expanded columns.

        Notes
        -----
        For a specific event ID you can expand the EventProperties values
        into their own columns using this function.
        You can do this for the whole data set but it will time-consuming
        and result in a lot of sparse columns in the output data frame.

        """
        if (
            not self._last_result or self._last_result.all_events is None
        ):  # type: ignore
            print(
                "Please use 'run()' to fetch the data before using this method.",
                "\nThen call 'expand_events()'",
            )
            return None
        return _parse_eventdata(
            event_data=self._last_result.all_events,  # type: ignore
            event_ids=event_ids,
        )


# %%
# Get Windows Security Events
def _get_win_security_events(qry_prov, host_name, timespan):
    nb_data_wait("SecurityEvent")

    all_events_df = qry_prov.WindowsSecurity.list_host_events(
        timespan,
        host_name=host_name,
        add_query_items="| where EventID != 4688 and EventID != 4624",
    )

    # Create a pivot of Event vs. Account
    win_events_acc = all_events_df[["Account", "Activity", "TimeGenerated"]].copy()
    win_events_acc = win_events_acc.replace("-\\-", "No Account").replace(
        {"Account": ""}, value="No Account"
    )
    win_events_acc["Account"] = win_events_acc.apply(
        lambda x: x.Account.split("\\")[-1], axis=1
    )
    event_pivot_df = (
        pd.pivot_table(
            win_events_acc,
            values="TimeGenerated",
            index=["Activity"],
            columns=["Account"],
            aggfunc="count",
        )
        .fillna(0)
        .reset_index()
    )
    return all_events_df, event_pivot_df


@set_text(docs=_CELL_DOCS, key="display_event_pivot")
def _display_event_pivot(event_pivot):
    display(
        event_pivot.style.applymap(lambda x: "color: white" if x == 0 else "")
        .applymap(
            lambda x: "background-color: lightblue"
            if not isinstance(x, str) and x > 0
            else ""
        )
        .set_properties(subset=["Activity"], **{"width": "400px", "text-align": "left"})
        .highlight_max(axis=1)
        .hide_index()
    )


# %%
# Extract event details from events
SCHEMA = "http://schemas.microsoft.com/win/2004/08/events/event"


def _parse_event_data_row(row):
    try:
        xdoc = ElementTree.fromstring(row.EventData)
        col_dict = {
            elem.attrib["Name"]: elem.text for elem in xdoc.findall(f"{{{SCHEMA}}}Data")
        }
        reassigned = set()
        for key, val in col_dict.items():
            if key in row and not row[key]:
                row[key] = val
                reassigned.add(key)
        if reassigned:
            for key in reassigned:
                col_dict.pop(key)
        return col_dict
    except (ParseError, TypeError):
        return None


def _expand_event_properties(input_df):
    # For a specific event ID you can explode the EventProperties values
    # into their own columns using this function. You can do this for
    # the whole data set but it will result
    # in a lot of sparse columns in the output data frame.
    exp_df = input_df.apply(lambda x: pd.Series(x.EventProperties), axis=1)
    return (
        exp_df.drop(set(input_df.columns).intersection(exp_df.columns), axis=1)
        .merge(
            input_df.drop("EventProperties", axis=1),
            how="inner",
            left_index=True,
            right_index=True,
        )
        .replace("", np.nan)  # these 3 lines get rid of blank columns
        .dropna(axis=1, how="all")
        .fillna("")
    )


@set_text(docs=_CELL_DOCS, key="parse_eventdata")
def _parse_eventdata(event_data, event_ids: Optional[Union[int, Iterable[int]]] = None):
    if event_ids:
        if isinstance(event_ids, int):
            event_ids = [event_ids]
        src_event_data = event_data[event_data["EventID"].isin(event_ids)].copy()
    else:
        src_event_data = event_data.copy()

    # Parse event properties into a dictionary
    nb_markdown("Parsing event datamsticnb.")
    src_event_data["EventProperties"] = src_event_data.apply(
        _parse_event_data_row, axis=1
    )
    return _expand_event_properties(src_event_data)


# %%
# Account management events
def _extract_acct_mgmt_events(event_data):
    # Get a full list of Windows Security Events

    w_evt = pkgutil.get_data("msticpy", f"resources{os.sep}WinSecurityEvent.json")
    win_event_df = pd.read_json(w_evt)

    # Create criteria for events that we're interested in
    acct_sel = win_event_df["subcategory"] == "User Account Management"
    group_sel = win_event_df["subcategory"] == "Security Group Management"
    schtask_sel = (win_event_df["subcategory"] == "Other Object Access Events") & (
        win_event_df["description"].str.contains("scheduled task")
    )

    event_list = win_event_df[acct_sel | group_sel | schtask_sel]["event_id"].to_list()
    # Add Service install event
    event_list.append(7045)
    return event_data[event_data["EventID"].isin(event_list)]


def _create_acct_event_pivot(account_event_data):
    # Create a pivot of Event vs. Account
    if account_event_data.empty:
        return None
    win_events_acc = account_event_data[["Account", "Activity", "TimeGenerated"]].copy()
    win_events_acc = win_events_acc.replace("-\\-", "No Account").replace(
        {"Account": ""}, value="No Account"
    )
    win_events_acc["Account"] = win_events_acc.apply(
        lambda x: x.Account.split("\\")[-1], axis=1
    )

    event_pivot_df = (
        pd.pivot_table(
            win_events_acc,
            values="TimeGenerated",
            index=["Activity"],
            columns=["Account"],
            aggfunc="count",
        )
        .fillna(0)
        .reset_index()
    )
    return event_pivot_df


@set_text(docs=_CELL_DOCS, key="display_acct_event_pivot")
def _display_acct_event_pivot(event_pivot_df):
    display(
        event_pivot_df.style.applymap(lambda x: "color: white" if x == 0 else "")
        .applymap(
            lambda x: "background-color: lightblue"
            if not isinstance(x, str) and x > 0
            else ""
        )
        .set_properties(subset=["Activity"], **{"width": "400px", "text-align": "left"})
        .highlight_max(axis=1)
        .hide_index()
    )


@set_text(docs=_CELL_DOCS, key="display_acct_mgmt_timeline")
def _display_acct_mgmt_timeline(acct_event_data):
    # Plot events on a timeline
    return nbdisplay.display_timeline(
        data=acct_event_data,
        group_by="EventID",
        source_columns=["Activity", "Account"],
        legend="right",
    )


---
# More Info

## msticpy
- Documentation - https://msticpy.readthedocs.io
- GitHub - https://github.com/microsoft/msticpy
- PyPI - https://pypi.org/project/msticpy/

## msticnb - Notebooklets
- GitHub - https://github.com/microsoft/msticnb

## Notebooks
- Azure-Sentinel-Notebooks - https://github.com/Azure/Azure-Sentinel-Notebooks
- Binder-able demo - https://github.com/Azure/Azure-Sentinel-Notebooks/tree/master/nbdemo

--
## Network Flow Notebooklet

In [24]:
nb.init(
    "LocalData",
    LocalData_data_paths=["e:\\src\\msticnb\\msticnb\\tests\\testdata"],
    LocalData_query_paths=["e:\\src\\msticnb\\msticnb\\tests\\testdata"],
)
flow_summary = nb.nblts.azsent.network.NetworkFlowSummary()
flow_result = flow_summary.run(value="MSTICAlertsWin1", timespan=TimeSpan(time_selector=time_span))

e:\src\microsoft\msticpy\msticpy\data\query_store.py:172: UserWarning: e:\src\msticnb\msticnb\tests\testdata\msticpyconfig-test.yaml is not a valid query definition file.
  warnings.warn(f"{file_path} is not a valid query definition file.")


Please wait. Loading Kqlmagic extension...


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

...

Loading BokehJS ...

Loading BokehJS ...

,source,dest,L7Protocol,FlowDirection,TotalAllowedFlows
0,10.0.3.5,13.107.4.50,http,O,2
1,10.0.3.5,13.65.107.32,https,O,9
2,10.0.3.5,13.67.143.117,https,O,2
3,10.0.3.5,13.71.172.128,https,O,16
4,10.0.3.5,13.71.172.130,https,O,29
5,10.0.3.5,134.170.58.123,https,O,5
6,10.0.3.5,20.38.98.100,https,O,1
7,10.0.3.5,204.79.197.200,https,O,5
8,10.0.3.5,23.48.36.47,http,O,2
9,10.0.3.5,40.124.45.19,https,O,9


Getting data from Whois...
..................

,DestASN,SourceASN,TotalAllowedFlows,L7Protocols,source_ips,dest_ips
0,"AKAMAI-ASN1, EU",No ASN Information for IP type: Private,2.0,[http],[10.0.3.5],[23.48.36.47]
1,"EDGECAST, US",No ASN Information for IP type: Private,6.0,"[https, http]",[10.0.3.5],"[72.21.81.200, 72.21.91.29]"
2,"MICROSOFT-CORP-MSN-AS-BLOCK, US",No ASN Information for IP type: Private,114.0,"[http, https, ntp]",[10.0.3.5],"[13.107.4.50, 13.65.107.32, 13.67.143.117, 13.71.172.128, 13.71.172.130, 134.170.58.123, 20.38.9..."


In [25]:
flow_summary.select_asns()

None does not appear to be an IPv4 or IPv6 address


In [26]:
flow_summary.lookup_ti_for_asn_ips()
flow_summary.show_selected_asn_map()

Getting data from Threat Intelligence...


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Warning - query did not complete successfully.
Unknown response from provider: None


AttributeError: 'NetworkFlowResult' object has no attribute 'merge'

In [32]:
from msticnb.nb_metadata import NBMetaData
newmd = eval(repr(nb.nblts.azsent.host.WinHostEvents.metadata))
newmd == nb.nblts.azsent.host.WinHostEvents.metadata

True

In [3]:
nb.nblts.azsent.host.WinHostEvents.import_cell()

AttributeError: type object 'WinHostEvents' has no attribute '__file__'

In [54]:
from msticnb.nb.azsent.host.host_summary import _CELL_DOCS

str(_CELL_DOCS)

metadata_repr = repr(nb.nblts.azsent.host.WinHostEvents.metadata)
metadata_repr = metadata_repr.replace("NBMetaData", "nb_metadata.NBMetaData")

In [4]:
from msticnb.nb.azsent.host import host_summary
host_summary.__file__
with open(host_summary.__file__, "r") as mod_file:
    mod_text = mod_file.read()

repl_text = "_CLS_METADATA, _CELL_DOCS = nb_metadata.read_mod_metadata(__file__, __name__)"
docs_repl = f"_CELL_DOCS = {str(_CELL_DOCS)}\n"
docs_repl = f"{docs_repl}\n_CLS_METADATA = {metadata_repr}"
print(mod_text.replace(repl_text, docs_repl)[1000:3000])

NameError: name '_CELL_DOCS' is not defined

In [6]:
nb.nblts.azsent.host.WinHostEvents.__module__

'msticnb.nb.azsent.host.win_host_events'